<a href="https://colab.research.google.com/github/nxxk23/PROJECT/blob/main/Model/ModelSVC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install shap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 538.2/538.2 kB 8.6 MB/s eta 0:00:00


In [3]:
!pip install scikeras

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold, cross_val_score
from sklearn.preprocessing import StandardScaler

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from keras.optimizers import Adam
from scikeras.wrappers import KerasRegressor

from imblearn.over_sampling import SMOTE
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

import shap


In [5]:
bi3 = pd.read_excel('/content/drive/MyDrive/ckdresearch2023/Assign/Assign8 : SplitFold /binary_3_Group_5_Fold.xlsx')
bi3 = bi3.iloc[:, 1:]

In [6]:
bi3['NATNL'] = bi3['NATNL'].replace({0: 'Asian', 1: 'Asian', 3: 'Asian', 2: 'Non-Asian'})

In [7]:
bi3 = bi3.sort_values(['PatientUID', 'PatientVisitUID'])
bi3['TimeDiff'] = bi3.groupby('PatientUID')['LabCwhen'].diff().dt.days / 365
bi3['TimeDiff'] = bi3['TimeDiff'].fillna(0)

In [8]:
bi3['time_t1'] = pd.to_datetime(bi3['time_t1'])

In [9]:
columns_to_encode = ['VisitLocation', 'Group', 'NATNL']

# Apply label encoding using map
for column in columns_to_encode:
    encoding_map = {category: i for i, category in enumerate(bi3[column].unique())}
    bi3[column] = bi3[column].map(encoding_map)

In [10]:
def categorize_eGFR(eGFR_value):
    if eGFR_value >= 90:
        return 0
    elif eGFR_value >= 60:
        return 1
    elif eGFR_value >= 30:
        return 2
    elif eGFR_value >= 15:
        return 3
    else:
        return 4

bi3['Stage'] = bi3['eGFR_t1'].apply(categorize_eGFR)

# Split

In [11]:
X = bi3.drop(['Slopes', 'PatientUID', 'PatientVisitUID', 'eGFR_t1', 'time_t1', 'StartDTTM', 'EndDttm',
               'LabCwhen', 'LabLocation', 'Fold'], axis=1)
y = bi3['Slopes']

In [12]:

num_folds = 5
kf = KFold(n_splits=num_folds, shuffle=True, random_state=None)
X = bi3.drop(['Slopes', 'PatientUID', 'PatientVisitUID', 'eGFR_t1', 'time_t1', 'StartDTTM', 'EndDttm',
               'LabCwhen', 'LabLocation', 'Fold'], axis=1)
y = bi3['Slopes']
all_folds = bi3['Fold'].unique()
X_train_scaled_list_bi3 = []
y_train_list_bi3 = []
X_test_scaled_list_bi3 = []
y_test_list_bi3 = []
for train_folds, test_fold in kf.split(all_folds):
    train_folds = all_folds[train_folds]
    val_fold = all_folds[test_fold][0]
    train_indices = bi3['Fold'].isin(train_folds)
    test_indices = bi3['Fold'] == val_fold
    X_train, X_test = X[train_indices], X[test_indices]
    y_train, y_test = y[train_indices], y[test_indices]
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)
    X_train_scaled_list_bi3.append(X_train_scaled)
    y_train_list_bi3.append(y_train)
    X_test_scaled_list_bi3.append(X_test_scaled)
    y_test_list_bi3.append(y_test)
for fold, (X_train_scaled_fold, y_train_fold, X_test_scaled_fold, y_test_fold, train_fold_indices) in enumerate(
        zip(X_train_scaled_list_bi3, y_train_list_bi3, X_test_scaled_list_bi3, y_test_list_bi3, kf.split(all_folds))
):
    train_folds, test_fold = train_fold_indices
    print(f"Training set size: {X_train_scaled_fold.shape}")
    print(f"Test set size: {X_test_scaled_fold.shape}")
    print(f"Folds in training set: {train_folds.tolist()}")
    print(f"Folds in test set: {test_fold}")
    print("---\n")


Training set size: (42027, 122)
Test set size: (10593, 122)
Folds in training set: [0, 1, 2, 3]
Folds in test set: [4]
---

Training set size: (41978, 122)
Test set size: (10642, 122)
Folds in training set: [0, 1, 2, 4]
Folds in test set: [3]
---

Training set size: (42184, 122)
Test set size: (10436, 122)
Folds in training set: [1, 2, 3, 4]
Folds in test set: [0]
---

Training set size: (42274, 122)
Test set size: (10346, 122)
Folds in training set: [0, 2, 3, 4]
Folds in test set: [1]
---

Training set size: (42017, 122)
Test set size: (10603, 122)
Folds in training set: [0, 1, 3, 4]
Folds in test set: [2]
---



#SVC

In [15]:
from sklearn.svm import SVC
from sklearn import svm

In [17]:
binary3_X = bi3.drop(['Group', 'PatientUID', 'PatientVisitUID', 'eGFR_t1', 'time_t1', 'StartDTTM', 'EndDttm',
               'LabCwhen', 'LabLocation','Fold','Slopes'], axis=1)
binary3_y = bi3['Group']


binary3_label_encoder = LabelEncoder()
bi3['Group'] = binary3_label_encoder.fit_transform(bi3['Group'])

num_folds = 5
kf = KFold(n_splits=num_folds, shuffle=True, random_state=None)
binary3_all_folds = bi3['Fold'].unique()

xgb_metrics_list = []
binary3_accuracy_per_fold = []
binary3_conf_mat_list = []
binary3_precision_per_fold = []
binary3_recall_per_fold = []
binary3_f1_per_fold = []
binary3_loss_per_fold = []


In [18]:
for fold, (train_folds, test_fold) in enumerate(kf.split(binary3_all_folds)):
    train_folds = binary3_all_folds[train_folds]
    test_fold = binary3_all_folds[test_fold][0]
    train_indices = bi3['Fold'].isin(train_folds)
    test_indices = bi3['Fold'] == test_fold

    binary3_X_train, binary3_X_test = binary3_X[train_indices], binary3_X[test_indices]
    binary3_y_train, binary3_y_test = binary3_y[train_indices], binary3_y[test_indices]

    # Scale
    binary3_scaler = StandardScaler()
    binary3_X_train_scaled = binary3_scaler.fit_transform(binary3_X_train)
    binary3_X_test_scaled = binary3_scaler.transform(binary3_X_test)

    # Create and train XGBoost model using best parameters
    model_svc = svm.SVC(decision_function_shape='ovo')
    model_svc.fit(binary3_X_train_scaled, binary3_y_train)

    # Make predictions on the test set
    binary3_y_pred = model_svc.predict(binary3_X_test_scaled)

    # Evaluate performance
    binary3_conf_mat = confusion_matrix(binary3_y_test, binary3_y_pred)
    binary3_conf_mat_list.append(binary3_conf_mat)
    binary3_accuracy = accuracy_score(binary3_y_test, binary3_y_pred)
    binary3_precision = precision_score(binary3_y_test, binary3_y_pred, average='weighted')
    binary3_recall = recall_score(binary3_y_test, binary3_y_pred, average='weighted')
    binary3_f1 = f1_score(binary3_y_test, binary3_y_pred, average='weighted')

    binary3_accuracy_per_fold.append(binary3_accuracy)
    binary3_precision_per_fold.append(binary3_precision)
    binary3_recall_per_fold.append(binary3_recall)
    binary3_f1_per_fold.append(binary3_f1)

    xgb_metrics_list.append({
        'Fold': fold + 1,
        'Accuracy': binary3_accuracy_per_fold,
        'Precision': binary3_precision_per_fold,
        'Recall': binary3_recall_per_fold,
        'F1 Score': binary3_f1_per_fold,
    })

In [19]:
import pandas as pd

# Create a DataFrame for performance metrics of each fold
fold_data = {
    "Fold": [f"Fold {fold+1}" for fold in range(len(binary3_accuracy_per_fold))],
    "Accuracy (%)": [accuracy * 100 for accuracy in binary3_accuracy_per_fold],
    "Precision": binary3_precision_per_fold,
    "Recall": binary3_recall_per_fold,
    "F1-Score": binary3_f1_per_fold
}

# Create DataFrame for fold performance
fold_performance_df = pd.DataFrame(fold_data).set_index("Fold")


# Calculate overall performance metrics
overall_metrics = {
    "Accuracy (%)": sum(binary3_accuracy_per_fold) / len(binary3_accuracy_per_fold) * 100,
    "Precision": sum(binary3_precision_per_fold) / len(binary3_precision_per_fold),
    "Recall": sum(binary3_recall_per_fold) / len(binary3_recall_per_fold),
    "F1-Score": sum(binary3_f1_per_fold) / len(binary3_f1_per_fold)
}
# Create DataFrame for overall performance
overall_performance_df = pd.DataFrame(overall_metrics, index=["Overall"])

# Concatenate fold performance DataFrame with overall performance DataFrame
result_svc = pd.concat([fold_performance_df, overall_performance_df])

# Print the concatenated DataFrame
print("\nSVC Performances:")
result_svc


SVC Performances:


,Accuracy (%),Precision,Recall,F1-Score
Fold 1,61.501462,0.586453,0.615015,0.590117
Fold 2,62.033636,0.590952,0.620336,0.594375
Fold 3,61.172472,0.584185,0.611725,0.583467
Fold 4,61.766961,0.587628,0.617670,0.589652
Fold 5,62.131178,0.592696,0.621312,0.595007
Overall,61.721142,0.588383,0.617211,0.590523
